In [4]:
# import all of the necessary libraries 
from __future__ import division
import pandas as pd


In [5]:
# Get TPM using tpm function 
def counts_to_tpm(counts_matrix):
    data_cols = counts_matrix.columns[1:]
    rpk = counts_matrix[data_cols].divide((counts_matrix['Length']/1000), axis=0)
    sums_per_million = rpk.sum(axis=0)/(10**6)
    tpm = rpk.divide(sums_per_million)
    
    return tpm

In [6]:
# read in the positve_iDOGS file 
dogs = pd.read_table('positive_iDOGS.txt',comment='#',index_col=0)
dogs.head()

/home/wschrein/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


,Chr,Start,End,Strand,Length,/home/wschrein/scratch/nuevo/att_1/bams/Sample1Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample2Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample3Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample4Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample5Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample6Aligned.out.sam.sorted.bam
Geneid,,,,,,,,,,,
WBGene00022276,I,10403,10423,+,21,0,0,0,0,0,0
WBGene00022275,I,43723,43743,+,21,1,1,0,0,0,1
WBGene00044345,I,47462,47482,+,21,16,17,32,3,3,2
WBGene00021677,I,49909,49929,+,21,2,8,0,0,0,0
WBGene00000812,I,71848,71868,+,21,10,13,17,10,15,13


In [7]:
dogs = dogs[dogs['Strand']=="+"]

In [8]:
# rename columns to something more palatable 
dogs = dogs.rename(columns={'/home/wschrein/scratch/nuevo/att_1/bams/Sample1Aligned.out.sam.sorted.bam':'ctr_1',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample2Aligned.out.sam.sorted.bam':'ctr_3',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample3Aligned.out.sam.sorted.bam':'ctr_4',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample4Aligned.out.sam.sorted.bam':'hs_1',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample5Aligned.out.sam.sorted.bam':'hs_3',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample6Aligned.out.sam.sorted.bam':'hs_4'
                          })
dogs.head(1)

,Chr,Start,End,Strand,Length,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4
Geneid,,,,,,,,,,,
WBGene00022276,I,10403,10423,+,21,0,0,0,0,0,0


In [9]:
# make an info column of things that you might need later 
info=dogs.reset_index()
info=info[['Geneid','Chr','Start','End']]

In [10]:
# now get rid of them 
dogs.drop(['Chr', 'Start', 'End'], inplace=True, axis=1)

In [12]:
dogs_tpm = counts_to_tpm(dogs) # get the tpm for all of the reads 
dogs_tpm.drop('Length', axis=1, inplace=True) # no longer need the length column

dogs_tpm.head(1)

,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4
Geneid,,,,,,
WBGene00022276,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
data_cols = dogs_tpm.columns[0:]
data_cols

Index([u'ctr_1', u'ctr_3', u'ctr_4', u'hs_1', u'hs_3', u'hs_4'], dtype='object')

In [16]:
dogs_tpm_filterd = dogs_tpm.loc[dogs_tpm[data_cols].sum(axis=1) > 0] 

In [17]:
dogs_tpm_filterd = dogs_tpm_filterd.copy()  # copy to get rid of slicing error

# Calculate the Intergenic Ratio

In [18]:
dogs_tpm_filterd['ctrl_total'] = dogs_tpm_filterd[dogs_tpm_filterd.columns[0:3]].sum(axis=1)

In [19]:
dogs_tpm_filterd['hs_total'] = dogs_tpm_filterd[dogs_tpm_filterd.columns[3:6]].sum(axis=1)

In [20]:
dogs_tpm_filterd['ctrl_total'] = dogs_tpm_filterd['ctrl_total'] +1 # need to add one to each replicate 

In [21]:
dogs_tpm_filterd['ratio'] = dogs_tpm_filterd.hs_total / dogs_tpm_filterd.ctrl_total

In [22]:
dogs_tpm_filterd.head(1)

,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio
Geneid,,,,,,,,,
WBGene00022275,15.34731,20.620683,0.0,0.0,0.0,13.462212,36.967992,13.462212,0.364159


## Make it so that only two replicates need to be non-zero ###


In [28]:
# Make it so that only two replicates have to be non-zero

_1_3 = dogs_tpm_filterd[(dogs_tpm_filterd['hs_1']!=0) & (dogs_tpm_filterd['hs_3']!=0)]
_1_4 = dogs_tpm_filterd[(dogs_tpm_filterd['hs_1']!=0) & (dogs_tpm_filterd['hs_4']!=0)]
_3_4 = dogs_tpm_filterd[(dogs_tpm_filterd['hs_3']!=0) & (dogs_tpm_filterd['hs_4']!=0)]


In [29]:
df_concat = pd.concat([_1_3,_1_4,_3_4]) # now add them on top of each other 
df_concat.reset_index(inplace=True)
df_concat.drop_duplicates(inplace=True)

In [30]:
hs_filtered_dogs = df_concat
df_concat.head(1)

,Geneid,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio
0,WBGene00044345,245.556954,350.551603,593.504832,25.063704,55.635919,26.924423,1190.613389,107.624045,0.090394


# change ratio to greater than 2 and see what happens
# This ratio seems to be the best for our purposes

In [33]:
# get list of Genes with an intergenic ratio of greater than 2
# By Trial an error this seems to be a good ratio to filter out DoGs



hs_filtered_dogs = hs_filtered_dogs[hs_filtered_dogs['ratio']>2]

hs_filtered_dogs.reset_index(inplace=True)

# Maybe we should filter the things that are not protein coding genes out


In [37]:
# get the list of protein coding genes 
pcgs = pd.read_table('gtf_protein_simpified.csv',names=['Geneid'])
pcgs.head(1) # perfect; 

/home/wschrein/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


,Geneid
0,WBGene00010957


In [38]:
# we are most concerned with filtering out pcgs
# so merge with list of protein coding genes
# pcgs were determined by parsing gtf file for genes 
# annotated as pcgs

pcgs_merge = pd.merge(hs_filtered_dogs,pcgs,how='inner',on='Geneid')

# Can we filter further with Intron Retention Score

In [40]:
# import intron retention scores to further filter out
# this file is a shortened version of the Intron Retention Generated in 
# Figure1/Calculate_Intron_Retention_Score
# it has been mereged with HS up genes, so it only has scores for HS up genes


s2 = pd.read_csv('s2_scores.csv')
s2.rename(columns={'WB_ID':'Geneid', 'ratio':'IR_score'}, inplace=True)
s2.drop(["Unnamed: 0"], inplace=True, axis=1)


In [41]:
s2.head(1)

,Geneid,IR_score
0,WBGene00000002,0.064036


In [42]:
pcgs_merge_IR = pd.merge(pcgs_merge, s2, on='Geneid', how='inner')
len(pcgs_merge_IR)

In [43]:
cutoff_pcgs_merge_IR = pcgs_merge_IR[pcgs_merge_IR['IR_score']> 0.4]
len(cutoff_pcgs_merge_IR)

29

In [79]:
# Create list of genes with IJ ratio greater than2 and IR score greater than 0.4
# by examination with genome browser this seems to filter out most bad DoGs

up_filtered_plus = cutoff_pcgs_merge_IR[['Geneid']]

In [83]:
## This file is used in Figure1/D1. Filter_the_up_Genes.ipynb
## to filter out interfering DoG on the + strand

up_filtered_plus.to_csv('up_bad_plus_dogs.csv',index=None)